In [25]:
'''''
为实现车牌数字字母字符识别(10个数字+24个字母为数据集)

'''''
import numpy as np
import tensorflow as tf
import random
import os
import matplotlib.pyplot as plt

def read_and_decode(tfrecords_file, batch_size=25):
    '''read and decode tfrecord file, generate (image, label) batches
    Args:
        tfrecords_file: the directory of tfrecord file
        batch_size: number of images in each batch
    Returns:
        image: 4D tensor - [batch_size, width, height, channel]
        label: 1D tensor - [batch_size]
    '''
    print('start')
    batch_size = batch_size
    filename_queue = tf.train.string_input_producerroducer([tfrecords_file])
    
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    img_features = tf.parse_single_example(
                                        serialized_example,
                                        features={
                                               'label': tf.FixedLenFeature([], tf.int64),
                                               'image_raw': tf.FixedLenFeature([], tf.string),
                                               })
    image = tf.decode_raw(img_features['image_raw'], tf.uint8)
    image = tf.reshape(image, [20, 36])
    image = tf.cast(image,tf.float32)*(1./255)
    label = tf.cast(img_features['label'], tf.int32)    
    image_batch, label_batch = tf.train.batch([image,label], batch_size = batch_size, capacity=10000)
    with tf.Session() as sess:
        i = 0

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        
        try:
            while not coord.should_stop() and i < 1:
                # just plot one batch size 
                
                image, label = sess.run([image_batch, label_batch])
                #plot_images(image, label)
                # image = image.reshape(784)
                i += 1
                
        except tf.errors.OutOfRangeError:
            print('done!')
        finally:
            coord.request_stop()
    coord.join(threads)
    w ,= label.shape
    # print(s)
    b = np.empty((w,36))
    for i in range(w):
        for j in range(36):
            if j==label[i]:
                b[i][j]=1.
            else:
                b[i][j]=0.
    
    s,d,f=image.shape
    images=np.empty((s,d*f))
    for q in range(s):
        c=image[q]
        images[q]=c.reshape(d*f)
        # image=images
        print('ok')
    return images, b



[X_train, y_train] = read_and_decode('plate_train.tfrecords', 50)
print(X_train.shape)
print(y_train.shape)
[X_val, y_val] = read_and_decode('plate_validation.tfrecords', 3475)



def chooseone(image, label, batchsize):
    im = np.empty((batchsize, 2160))
    la = np.empty((batchsize, 36))
    for i in range(batchsize):
        a = random.randint(0, 3474)
        im[i] = image[a]
        la[i] = label[a]
    return im, la


# 权值初始化
def weight_variable(shape):
    # 用正态分布来初始化权值
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    # 本例中用relu激活函数，所以用一个很小的正偏置较好
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


# 定义卷积层
def conv2d(x, W):
    # 默认 strides[0]=strides[3]=1, strides[1]为x方向步长，strides[2]为y方向步长
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


# pooling 层
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


X_ = tf.placeholder(tf.float32, [None, 2160])
y_ = tf.placeholder(tf.float32, [None, 36])

# 把X转为卷积所需要的形式
X = tf.reshape(X_, [-1, 48, 24, 1])
# 第一层卷积：3×3×1卷积核32个 [3，3，1，32],h_conv1.shape=[-1, 48, 24, 32],学习32种特征
W_conv1 = weight_variable([3, 3, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)

# 第一个pooling 层[-1, 48, 24, 32]->[-1, 24, 12, 32]
h_pool1 = max_pool_2x2(h_conv1)

# 第二层卷积：5×5×32卷积核64个 [3，3，32，64],h_conv2.shape=[-1, 24, 12, 64]
W_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# 第二个pooling 层,[-1, 24, 12, 64]->[-1, 14, 6, 64] 
h_pool2 = max_pool_2x2(h_conv2)

# 
W_conv3 = weight_variable([3, 3, 64, 96])
b_conv3 = bias_variable([96])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

# flatten层，[-1, 6, 3, 96]->[-1, 6*3*96],即每个样本得到一个6*3*96维的样本
h_pool2_flat = tf.reshape(h_pool3, [-1, 6*3*96])

# fc1
W_fc1 = weight_variable([6*3*96, 512])
b_fc1 = bias_variable([512])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout: 输出的维度和h_fc1一样，只是随机部分值被值为零
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 输出层
W_fc2 = weight_variable([512, 36])
b_fc2 = bias_variable([36])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# 1.损失函数：cross_entropy
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
# 2.优化函数：AdamOptimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 3.预测准确结果统计
# 预测值中最大值（１）即分类结果，是否等于原始标签中的（１）的位置。argmax()取最大值所在的下标
z = tf.argmax(y_conv, 1)
q = tf.arg_max(y_, 1)
correct_prediction = tf.equal(z, q)  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
test_acc_sum = tf.Variable(0.0)
batch_acc = tf.placeholder(tf.float32)
new_test_acc_sum = tf.add(test_acc_sum, batch_acc)
update = tf.assign(test_acc_sum, new_test_acc_sum)
saver = tf.train.Saver(max_to_keep=2)
# 定义了变量必须要初始化，或者下面形式
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(FLAGS.my_list, sess.graph)

    ckpt = tf.train.latest_checkpoint(FLAGS.my_list)
    step = 0
    if ckpt:
        check_point_path = '/home/r/mycar/cnn'  # 保存好模型的文件路径
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir=check_point_path)
        saver.restore(sess, ckpt.model_checkpoint_path)

    Y = np.zeros(3000)
    X_batch, y_batch = chooseone(X_val, y_val, 3000)
    Ytemp = y_conv.eval(feed_dict={X_: X_batch, keep_prob: 1.0})
    for i in range(3000):
        Y[i] = np.argmax(Ytemp[i])
    print("验证集正确率为 %g" % accuracy.eval(feed_dict={X_: X_batch, y_: y_batch, keep_prob: 1.0}))


for j in range(36):
    k = 0
    l = 0
    for i in range(3000):
        if np.argmax(y_batch[i]) == j:
            k = k+1
            if Y[i] == j:
                if np.argmax(y_batch[i]) == Y[i]:
                    l = l+1
    if j < 26:
        a = str(chr(ord('A') + j))
    elif j > 25:
        a = str(chr(ord('0') + j - 26))
    if k==0:
        print('样本中无%s 无法计算' % a)
    else:
        print('%s召回率为%g' % (a, (l/k)))



start
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Input to reshape is a tensor with 2160 values, but the requested shape has 1152
	 [[Node: Reshape = Reshape[T=DT_UINT8, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](DecodeRaw, Reshape/shape)]]
done!


InvalidArgumentError: Input to reshape is a tensor with 2160 values, but the requested shape has 1152
	 [[Node: Reshape = Reshape[T=DT_UINT8, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](DecodeRaw, Reshape/shape)]]